In [ ]:

import sqlite3
import os,json
import torch
entity_embeddings = torch.load(r'D:\Work\Data\GSN_Projects\datasource\sgmkg\dataset\data_entity.pth', map_location=torch.device('cpu'))

def get_similarity(embedding1, embedding2):
    sim = torch.cosine_similarity(embedding1, embedding2, dim=1)
    return sim.item()

db_path = r'PATH TO DATABASE FROM DATA_PREPARATION'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


cursor.execute("SELECT id, script_name FROM script")
scripts = cursor.fetchall()

# traverse each script
for script_id, script_name in scripts:
    cur_nodes = []
    cur_edges = []

    cur_nodes.append({"script_id":script_id, "name": script_name, "entity_type":"modeling task"})
    cursor.execute("SELECT id, step_name, short_name, description FROM step WHERE script_id = ?", (script_id,))
    steps_info = cursor.fetchall()
    current_step_ids = [step[0] for step in steps_info]
    current_step_names = [step[1] for step in steps_info]
    all_steps = set(current_step_ids)
    for id, step_name, short_name, description in steps_info:
        cur_nodes.append({"step_id": id,"name": step_name, "short_name":short_name, 
                          "description":description,"entity_type":"functional_step"})

    # Check step_workflow table，obtain output_step_id and input_step_id basede on script_id
    cursor.execute("SELECT output_step_id, input_step_id FROM step_workflow WHERE script_id = ?", (script_id,))
    workflows = cursor.fetchall()

    io_nodes = {}  ## use id as key,value:0 unused；1 used
    # prepare all potential io-data
    for step_id in current_step_ids:
        cursor.execute("SELECT id from io_data WHERE step_id = ?", (step_id,))
        io_data_ids = cursor.fetchall()
        for io_data_id in io_data_ids:
            io_nodes[io_data_id[0]] = 0

    used_steps = set()
    # traverse each workflow and obtain corresopnding step_name
    for output_id, input_id in workflows:
        
        cursor.execute("SELECT step_name FROM step WHERE id = ?", (output_id,))
        output_step_name = cursor.fetchone()
        cursor.execute("SELECT step_name FROM step WHERE id = ?", (input_id,))
        input_step_name = cursor.fetchone()
        used_steps.add(output_id)
        used_steps.add(input_id)
        ## save invoke relationship between functional steps
        if output_step_name and input_step_name:
            output_step_name = output_step_name[0]
            input_step_name = input_step_name[0]
            cur_edges.append({"output_id":output_id,"output": output_step_name, 
                              "input_id":input_id,"input": input_step_name, "relation":"invoke"})
            # take  io_data from each output functional step as well as io_data from input functional step and match them usning similarity
            cursor.execute("SELECT id, data FROM io_data WHERE step_id = ? and data_type = ?", (output_id,"output"))
            output_data = cursor.fetchall()
            output_data_ids = [data[0] for data in output_data]
            output_data_names = [data[1] for data in output_data]


            cursor.execute("SELECT id, data FROM io_data WHERE step_id = ? and data_type = ?", (input_id,"input"))
            input_data = cursor.fetchall()
            input_data_ids = [data[0] for data in input_data]
            input_data_names = [data[1] for data in input_data]

            # It is possible that steps are connected, but no specific io-data is found. 
            # Use a boolean optimization: if no connection is found, use the current output as a new object and add it to the input.
            is_connected = False
            for i, output_data_id in enumerate(output_data_ids):
                cur_output_emb = entity_embeddings[output_data_id]
                for j, input_data_id in enumerate(input_data_ids):
                    cur_input_emb = entity_embeddings[input_data_id]
                    similarity = get_similarity(cur_output_emb, cur_input_emb)
                    # If the similarity between an output and an input is very high or even identical, it indicates that this input serves as the output. 
                    # Merge these two output-input pairs into a single node.
                    if (similarity >=0.8):
                        io_nodes[output_data_id] = 1
                        io_nodes[input_data_id] = 1
                        # This merged node is the entity required for the final subgraph, and it needs to establish connections with predecessor and successor nodes as edges.
                        combined_entity = {"name": output_data_names[i]+"||"+input_data_names[j], "data_id":[output_data_id, input_data_id],"entity_type":"intermediate"}
                        cur_nodes.append(combined_entity)
                        new_edges = [{"output_step_id":output_id,"output": output_step_name, "input_data_id": combined_entity["data_id"],"input": combined_entity["name"], "relation":"transfer"},
                                     {"output_data_id":combined_entity["data_id"],"output": combined_entity["name"], "input_step_id":input_id,"input": input_step_name, "relation":"transfer"}]
                        cur_edges.extend(new_edges)
                        is_connected = True
            if not is_connected and len(output_data_ids) == 1:
                ## If two steps are connected but no suitable io-data is found, merge the predecessor node's output as the successor node's input (only in the case where there is exactly one output)
                cur_edges.extend([{"output_step_id":output_id,"output": output_step_name,  "input_data_id":  output_data_id, "input": output_data_names[0], "relation":"transfer"},
                                  {"output_data_id": output_data_id, "output": output_data_names[0], "input_step_id":input_id, "input": input_step_name, "relation":"transfer"}])
                io_nodes[output_data_ids[0]] = 0

    for k,v in io_nodes.items():
        if v == 0:
            # add ununsed node
            cursor.execute("SELECT step_id, data, data_type FROM io_data WHERE id = ? ", (k,))
            info = cursor.fetchone()
            step_id = info[0]
            data = info[1]
            data_type = info[2]
            cursor.execute("SELECT step_name FROM step WHERE id = ?", (step_id,))
            step_name =  cursor.fetchone()[0]
            if data_type == 'input':
                cur_nodes.append({"data_id":k, "name": data, "entity_type":"input"})
                cur_edges.append({"output_data_id": k, "output": data, "input_step_id":step_id, "input": step_name, "relation":"transfer"})
            else:
                cur_nodes.append({"data_id":k, "name": data, "entity_type":"output"})
                cur_edges.append({"output_step_id": step_id, "output": step_name, "input_data_id":k, "input": data, "relation":"transfer"})
    
    rest_steps = all_steps - used_steps
    for step_id in rest_steps:
        cursor.execute("SELECT short_name FROM step WHERE id = ?", (step_id,))
        short_name = cursor.fetchone()[0]
        cur_nodes.append({"step_id": step_id, "name": short_name, "entity_type":"functional_step"})
        

    for edge in cur_edges:
        if edge['relation'] !='transfer':
            continue
        if 'output_step_id' in edge:
            node_id = edge['input_data_id']
            entity_type = 'output'
        elif 'output_data_id' in edge:
            node_id = edge['output_data_id']
            entity_type = 'input'

        temp_node_list = []
        for node in cur_nodes:
            if node['entity_type'] in ['input', 'output', 'intermediate']:
                temp_node_list.append(node['data_id'])
        if node_id not in temp_node_list:
            print(script_name, "find unsaved node")
            cur_nodes.append({ "data_id": node_id ,"name": edge['output'],"entity_type": entity_type})

    data = {"nodes": cur_nodes, "edges": cur_edges}

    
    folder_path = r"preprocessing"
    file_path = os.path.join(folder_path, os.path.splitext(script_name)[0]+'.json')
    # save connection in JSON file to extract triples
    with open(file_path, 'w', encoding='utf-8-sig') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

conn.close()

In [ ]:
import os
import json
import torch

# Define folder paths
input_folder = r"preprocessing"
output_folder = r"triple"

def get_similarity(embedding1, embedding2):
    # Compute cosine similarity between two embedding vectors
    sim = torch.cosine_similarity(embedding1, embedding2, dim=1)
    return sim.item()

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Iterate over each JSON file in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):
        input_file_path = os.path.join(input_folder, file_name)
        # Read JSON file content
        with open(input_file_path, 'r', encoding='utf-8-sig') as json_file:
            data = json.load(json_file)

        # Reconstruct node IDs
        for node in data['nodes']:
            if node['entity_type'] == 'modeling task':
                node['script_id'] = 'script_' + str(node['script_id'])
            elif node['entity_type'] == 'functional_step':
                node['step_id'] = 'step_' + str(node['step_id'])
            else:
                node['data_id'] = 'iodata_' + str(node['data_id'])

        # Update edge IDs
        for edge in data['edges']:
            if edge['relation'] == 'invoke':
                edge['output_id'] = 'step_' + str(edge['output_id'])
                edge['input_id'] = 'step_' + str(edge['input_id'])
            elif edge['relation'] == 'transfer':
                if 'output_data_id' in edge:
                    edge['output_data_id'] = 'iodata_' + str(edge['output_data_id'])
                if 'input_step_id' in edge:
                    edge['input_step_id'] = 'step_' + str(edge['input_step_id'])
                if 'output_step_id' in edge:
                    edge['output_step_id'] = 'step_' + str(edge['output_step_id'])
                if 'input_data_id' in edge:
                    edge['input_data_id'] = 'iodata_' + str(edge['input_data_id'])

        remove_node_ids = []
        # Merge similar input nodes and remove duplicates
        for i, node in enumerate(data['nodes']):
            if node['entity_type'] == "input" and node['data_id']:
                current_id = node['data_id']
                index = int(current_id.split('_')[-1])
                current_embedding = entity_embeddings[index]
                # Compare with other input nodes
                for j, other_node in enumerate(data['nodes']):
                    if j > i and other_node['entity_type'] == "input":
                        cur_index = int(other_node['data_id'].split('_')[-1])
                        other_embedding = entity_embeddings[cur_index]
                        similarity = get_similarity(current_embedding, other_embedding)
                        if similarity > 0.8:
                            id_to_replace = other_node['data_id']
                            remove_node_ids.append(id_to_replace)
                            # Update edge references
                            for edge in data['edges']:
                                if 'output_data_id' in edge and edge['output_data_id'] == id_to_replace:
                                    edge['output_data_id'] = current_id
                                if 'input_data_id' in edge and edge['input_data_id'] == id_to_replace:
                                    edge['input_data_id'] = current_id

        print(f"Nodes to be removed: {remove_node_ids}")
        data['nodes'] = [node for node in data['nodes'] if str(node.get('data_id')) not in remove_node_ids]

        # Initialize sets to track existing data_id and step_id
        seen_data_ids = set()
        seen_step_ids = set()

        # Initialize lists to store unique and removed nodes
        unique_nodes = []
        removed_nodes = []

        # Check and remove duplicate nodes
        for node in data['nodes']:
            data_id = node.get('data_id')
            step_id = node.get('step_id')
            
            # Check if data_id or step_id already exists
            if (data_id and data_id in seen_data_ids) or (step_id and step_id in seen_step_ids):
                removed_nodes.append(node)
            else:
                unique_nodes.append(node)
                if data_id:
                    seen_data_ids.add(data_id)
                if step_id:
                    seen_step_ids.add(step_id)

        # Update data['nodes'] with unique nodes
        data['nodes'] = unique_nodes
        print(f"Removed duplicate nodes: {removed_nodes}")

        # Add new edges
        modeling_task_ids = []
        for node in data['nodes']:
            if 'script_id' in node:
                modeling_task_ids.append(node['script_id'])
        
        for mt_id in modeling_task_ids:
            for node in data['nodes']:
                if node['entity_type'] in ["intermediate", "input", "output"]:
                    new_edge = {
                        "output_id": mt_id,
                        "input_id": node['data_id'],
                        "relation": "contain"
                    }
                    data['edges'].append(new_edge)
                elif node['entity_type'] in ["functional_step"]:
                    new_edge = {
                        "output_id": mt_id,
                        "input_id": node['step_id'],
                        "relation": "contain"
                    }
                    data['edges'].append(new_edge)

        # Output the reconstructed JSON file
        output_file_path = os.path.join(output_folder, file_name)
        with open(output_file_path, 'w', encoding='utf-8-sig') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)
        
        print(f"Processed and saved file: {output_file_path}")


In [ ]:
### save results to a new sgmkg.db
import os
import json
import sqlite3

# Database setup
db_path = "sgmkg.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS raw_entity (
        id TEXT PRIMARY KEY,
        entity_type TEXT,
        short_name TEXT,
        descr TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS raw_relation (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        output_entity_id INTEGER,
        input_entity_id INTEGER,
        relation_type TEXT
    )
''')
conn.commit()
# Folder paths
input_folder = r"triples"

# Process each JSON file in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):
        input_file_path = os.path.join(input_folder, file_name)
        
        # Read the JSON file
        with open(input_file_path, 'r', encoding='utf-8-sig') as json_file:
            data = json.load(json_file)
        
        print(file_name)
        # Process each node and insert into raw_entity table
        for node in data['nodes']:
            entity_type = node['entity_type']
            short_name = node['name'] if entity_type == "modeling task" else node.get('short_name', node['name'])
            descr = "" if entity_type == "modeling task" else node.get('description', node['entity_type'])
            if entity_type == 'modeling task':
                id = node['script_id']
            elif entity_type == 'functional_step':
                id = node['step_id']
            else:
                id = node['data_id']
            if entity_type not in ["modeling task", "functional_step"]:
                entity_type = "io_data"
            
            cursor.execute('''
                INSERT INTO raw_entity (id, entity_type, short_name, descr)
                VALUES (?, ?, ?, ?)
            ''', (id, entity_type, short_name, descr))
            
        
        # Process each edge and insert into raw_relation table
        for edge in data['edges']:
            relation_type = edge['relation']
            output_entity_id = None
            input_entity_id = None
            
            if relation_type == "contain" or relation_type == "invoke":
                output_entity_id = edge['output_id']
                input_entity_id = edge['input_id']
            else:
                if 'output_data_id' in edge:
                    output_entity_id = edge['output_data_id']
                    input_entity_id = edge['input_step_id']
                else:
                    output_entity_id = edge['output_step_id']
                    input_entity_id = edge['input_data_id']
            
            cursor.execute('''
                INSERT INTO raw_relation (output_entity_id, input_entity_id, relation_type)
                VALUES (?, ?, ?)
            ''', (output_entity_id, input_entity_id, relation_type))
        
        print(f"Save file: {input_file_path}")

# Commit changes and close the database connection
conn.commit()
conn.close()

print("Database update and JSON file processing complete.")


In [3]:

import os
import openai
import pandas as pd

os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

# 获取环境变量 OPENAI_API_KEY
openai.api_key = "sk-svcacct-heoxQU5w6spvUqtY3kai5vprDN-WR6Fd0dHsrZelCQLeT3BlbkFJY5ghj1UCRlyWnUsVAwQUJkCHhkH4UsZzjWs7IdvykbsA"
input_csv = r'D:\Work\Data\GSN_Projects\datasource\sgmkg\dataset\sample'

PROMPT = '''
Suppose you are a geospatial scientist, given is a sentence:

{}

You are asked to (1) give a short name for this sentence(less than 10 words); 
(2) If there is specific time scope or spatial location in the string, make them universal; 
(3) Based on your geospatial research knowledge, expand the abbriviation in the string if has. 
    For instance, NDVI -> normalized difference vegetation index.
(4) If not necessary to revise, keep the original sentence.

Just return the new short name for the sentence exactly without any other words.
'''

PROMPT_DESCR = '''
Suppose you are a geospatial scientist, given is a description:

{}

You are asked to (1) summary this description (less than 50 words), 
     just describe the functionality of this description, you don't need to start with "The purpose of this step is to ..."; 
(2) remove all sentence about API or Google earth engine elements.
(3) If there is specific time scope or spatial location in the string, make them universal; 
(4) Based on your geospatial research knowledge, expand the abbriviation in the string if has. 
    For instance, NDVI -> normalized difference vegetation index.
(4) If not necessary to revise, keep the original sentence.

Just return the new description exactly without any other words.
'''

def get_completion(prompt, model="gpt-4o-mini", temperature=0.2, messages=None):
    '''
    Retrieve completions from the OpenAI API, preserving conversation history.
    '''
    if messages is None:
        messages = []

    messages.append({"role": "user", "content": prompt})
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    response_content = response.choices[0].message.content
    messages.append({"role": "system", "content": response_content})
    return response_content, messages

# 定义文件路径
input_csv_path = r'D:\Work\Data\GSN_Projects\datasource\sgmkg\dataset\kg_raw\functional_step_entities.csv'
output_csv_path = r'D:\Work\Data\GSN_Projects\datasource\sgmkg\dataset\kg_raw\functional_step_entities_with_content.csv'
# 读取输入CSV文件
df = pd.read_csv(input_csv_path)

# 如果输出CSV文件不存在，则创建新的文件并写入列名
if not os.path.exists(output_csv_path):
    pd.DataFrame(columns=['id', 'short_name', 'content', 'descr', 'descr_content']).to_csv(output_csv_path, index=False)

# 读取输出CSV文件中的已有数据
df_output = pd.read_csv(output_csv_path)

# 获取output_csv_path中已有的id列表
existing_ids = df_output['id'].tolist()

# 过滤出input_csv_path中还不在output_csv_path中的行
df_to_process = df[~df['id'].isin(existing_ids)]

count = df_to_process.shape[0]
# 处理每个short_name
for _, row in df_to_process.iterrows():
    count2 = count2+1
    cur_id = row['id']
    short_name = row['short_name']
    descr = row['descr']
    
    # 创建当前的prompt
    cur_prompt = PROMPT.format(short_name)
    
    # 获取生成的content
    content, _ = get_completion(cur_prompt, model="gpt-4o-mini")

    # cur_prompt = PROMPT_DESCR.format(descr)
    #  # 获取生成的description
    # descr_content, _ = get_completion(cur_prompt, model="gpt-4o-mini")
    
    count = count -1
    # 检查id是否已经在输出文件中存在
    if cur_id in df_output['id'].values:
        # 更新已有行的content
        df_output.loc[df_output['id'] == cur_id, 'content'] = content
        # df_output.loc[df_output['id'] == cur_id, 'descr_content'] = descr_content
    else:
        # 新增行
        new_row = {'id': cur_id, 'short_name': short_name, 'content': content, 'descr': descr, 'descr_content': ''}
        df_output = df_output.append(new_row, ignore_index=True)
    
    # 追加更新到输出文件
    df_output.to_csv(output_csv_path, index=False)
    print(f"New content of {cur_id}: {content}, original content: {short_name}, {count} left")

print(f"数据已成功更新并保存到 {output_csv_path}")

New content of step_2136: Image Classification, original content: Classify Image, 26944 left
New content of step_2137: Image Export, original content: Export Image, 26943 left
New content of step_2138: Import Geographic Features, original content: Import Geographical Features, 26942 left
New content of step_2139: Show Geographic Features, original content: Display Geographical Features, 26941 left
New content of step_2140: Nighttime Lights Data Processing, original content: Import and Process Nighttime Lights Data, 26940 left
New content of step_2141: Visualize Processed Nighttime Illumination Data, original content: Display Processed Nighttime Lights Data, 26939 left
New content of step_2142: Export Processed Nighttime Illumination Data, original content: Export Processed Nighttime Lights Data, 26938 left
New content of step_2143: Library Import and Geometry Definition, original content: Importing libraries and defining geometries, 26937 left
New content of step_2144: Map Layering, or